In [9]:
import requests
import wptools
import re
# from bs4 import BeautifulSoup
import pandas as pd

In [2]:
page = wptools.page('Lebanese_Civil_War')
page.get_parse()

en.wikipedia.org (parse) Lebanese_Civil_War
en.wikipedia.org (imageinfo) File:Martyrs Square 1982.jpg
Lebanese Civil War (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Martyrs S...
  infobox: <dict(10)> conflict, partof, image, image_size, caption...
  iwlinks: <list(2)> https://commons.wikimedia.org/wiki/Category:L...
  pageid: 312905
  parsetree: <str(129430)> <root><template><title>short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Lebanese Civil War
  wikibase: Q208484
  wikidata_url: https://www.wikidata.org/wiki/Q208484
  wikitext: <str(107439)> {{short description|1975–1990 civil war ...
}


In [8]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "Lebanese_Civil_War",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('Lebanese Civil War')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
14 July Revolution
17 May Agreement
17 October Revolution
1860 conflict in Lebanon
1935 Yazidi revolt
1935–1936 Iraqi Shia revolts
1943 Barzani revolt
1947–1949 Palestine war
1947–48 Civil War in Mandatory Palestine
1948 Arab–Israeli War
1948 Palestinian exodus
1949 Armistice Agreements
1953 Iranian coup d'état
1958 Lebanon crisis
1959 Mosul uprising
1963 Syrian coup d'état
1964 Hama riot
1966 Syrian coup d'état
1967 Palestinian exodus
1967 Six Day War
1968 Israeli raid on Lebanon
1973 Yom Kippur War
1974–75 Shatt al-Arab conflict
1976 Arab League summit (Riyadh)
1977 Shia uprising in Iraq
1978 South Lebanon conflict
1979 Qatif Uprising
1979–1980 Shia uprising in Iraq
1980 Turkish coup d'état
1982 Beirut bombing
1982 Iranian diplomats kidnapping
1982 Lebanon War
1982 kidnapping of Iranian diplomats
1983 Beirut barracks bombing
1983 Beirut barracks bombings
1983 United States embassy bombing
1984 Sohmor massacre
1984 United States embassy annex bombing
1984 United States embassy 


Page 2
List of wars involving Israel
Litani River
Little Satan
Lod Airport massacre
London Conference of 1939
London Conference of 1946–1947
Ma'alot massacre
Ma'n dynasty
Maarouf Saad
Madrid Conference of 1991
Mafia
Mahmoud Ahmadinejad and Israel
Mahmud Barzanji revolts
Mahsum Korkmaz
Majid Shahriari
Malcolm H. Kerr
Mamluk Sultanate (Cairo)
Mandate for Syria and the Lebanon
Mandatory Palestine
Marada Brigade
Marada Movement
March 14 Alliance
March 2012 Gaza–Israel clashes
March 2017 Israel–Syria incident
March 8 Alliance
Marcia C. Inhorn
Marjayoun
Maronite
Martyrs' Square, Beirut
Marxist-Leninist
Masoud Alimohammadi
May 17 Agreement
May 2013 Rif Dimashq airstrikes
McMahon–Hussein Correspondence
Memory Box (film)
Menachem Begin
Metn
Michel Aoun
Middle Eastern theatre of World War I
Military of Lebanon
Mirdasid dynasty
Misgav Am hostage crisis
Mohammad Hussein Fadlallah
Mohsen Fakhrizadeh
Monte Melkonian
Moral authority
Morocco
Morrison–Grady Plan
Mostafa Ahmadi Roshan
Mothers' Bus atta

In [22]:
def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

def to_delete(s):
    return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s

def get_list_items(div):
    s = re.sub('{{.*?}}', '', div)
    s = re.sub('\'', '', s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
    s = re.sub('\n----', '<br>', s)
    s = re.sub('\n\*', '<br>', s)
    s = re.sub('<br />', '<br>', s)
    s = re.sub('<br/>', '<br>', s)
    s = re.sub('</ref>|ref||</ref>', '', s)
    s = re.sub('</ref>', '', s)
    s = re.sub('ref', '', s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     if '<br>' in s:
    s = s.split("<br>")
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
    s = [remove_file_instances(e).strip() for e in s]
    s = [e for e in s if not to_delete(e)]
    return s

In [23]:
df_political_party = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Relegion', 'Founder(s)',
                                          'National Affiliation', 'International Affiliation', 
                                           # 'Allies', 'Opponents',
                                           'Headquarter(s)'])

In [24]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        page.get_parse(show=False)

        if page.data['infobox'] is not None:

            if 'ideology' in page.data['infobox']:
                print(page)

                leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                if leaders_types == []:
                    pass
                else:
                    founders = ''
                    count = 0
                    for lt in leaders_types:
                        if count%2 == 0:
                            founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                        else:
                            founders += ''.join(get_list_items(page.data['infobox'][lt])) + ','
                        count += 1
                name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                headquarters = get_list_items(page.data['infobox']['headquarters']) if 'headquarters' in page.data['infobox'] else '-'
                ideology = get_list_items(page.data['infobox']['ideology'])
                position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'
                religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                national = get_list_items(page.data['infobox']['national']) if 'national' in page.data['infobox'] else '-'
                international = get_list_items(page.data['infobox']['international']) if 'international' in page.data['infobox'] else '-'


                print('-----------------')
                print('name: ', name)
                print('founders: ', founders)
                print('headquarters: ', headquarters)
                print('ideology: ', ideology)
                print('position: ', position)
                print('relegion: ', religion)
                print('national: ', national)
                print('international: ', international)
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 14 July Revolution
en.wikipedia.org (imageinfo) File:Abd al-Karim Qasim & Abd al-Sal...
en.wikipedia.org (parse) 17 May Agreement
en.wikipedia.org (parse) 17 October Revolution
en.wikipedia.org (imageinfo) File:Beirut Protests 2019.jpg
en.wikipedia.org (parse) 1860 conflict in Lebanon
en.wikipedia.org (imageinfo) File:DamasChristianQuarter1860.jpg
en.wikipedia.org (parse) 1935 Yazidi revolt
en.wikipedia.org (parse) 1935–1936 Iraqi Shia revolts
en.wikipedia.org (parse) 1943 Barzani revolt
en.wikipedia.org (parse) 1947–1949 Palestine war
en.wikipedia.org (parse) 1947–48 Civil War in Mandatory Palestine
en.wikipedia.org (imageinfo) File:Katamon.jpg
en.wikipedia.org (parse) 1948 Arab–Israeli War
en.wikipedia.org (imageinfo) File:Raising the Ink Flag at Umm Ras...
en.wikipedia.org (parse) 1948 Palestinian exodus
en.wikipedia.org (parse) 1949 Armistice Agreements
en.wikipedia.org (parse) 1953 Iranian coup d'état
en.wikipedia.org (imageinfo) File:Operationajax.jpg
en.

-----------------
name:  ['Fatah – The Revolutionary Council']
founders:  Abu Nidal:
headquarters:  -
ideology:  ['Palestinian nationalism', 'Anti-Zionism', 'Secularism']
position:  -
relegion:  -
national:  -
international:  -


en.wikipedia.org (parse) Abu Nidal Organization internal executions
en.wikipedia.org (parse) Achaemenid Phoenicia
en.wikipedia.org (parse) Aden Emergency
en.wikipedia.org (imageinfo) File:South Yemen in its region.svg
en.wikipedia.org (parse) Administrative divisions of Lebanon
en.wikipedia.org (parse) Adwan Rebellion
en.wikipedia.org (parse) Agreement on Disengagement between Israe...
en.wikipedia.org (parse) Agriculture in Lebanon
en.wikipedia.org (parse) Ahmed Barzani revolt
en.wikipedia.org (imageinfo) File:3.7inchHowitzerBarzanOperations...
en.wikipedia.org (parse) Ain es Saheb airstrike
en.wikipedia.org (imageinfo) File:Ein saheb strike.jpg
en.wikipedia.org (parse) Akkar
en.wikipedia.org (parse) Al-Aqsa Martyrs' Brigades


-----------------
name:  ['al-Aqsa Martyrs Brigades', 'Katāib Shuhadā al-Aqṣā']
founders:  Yasser Arafat (former)Marwan Barghouti (convicted)Zakaria Zubeidi (former)Naif Abu-SharahFadi Kafisha:
headquarters:  -
ideology:  ['Palestinian nationalism', 'Socialism', 'Anti-Zionism']
position:  -
relegion:  -
national:  -
international:  -


en.wikipedia.org (parse) Al-Mourabitoun


...

-----------------
name:  ['Army of the Holy War']
founders:  Abd al-Qadir al-Husayni, Hasan Salama:
headquarters:  ['Bir Zeit']
ideology:  ['Anti-Zionism']
position:  -
relegion:  -
national:  -
international:  -


en.wikipedia.org (parse) As-Sa'iqa


-----------------
name:  ['As-Saiqa']
founders:  Dr Mohammed Qeis:
headquarters:  -
ideology:  ['Pan-Arabism', 'Baathism']
position:  -
relegion:  -
national:  ['PLO', 'APF']
international:  -


en.wikipedia.org (parse) Assaf dynasty
en.wikipedia.org (parse) Assassination of Bachir Gemayel
en.wikipedia.org (imageinfo) File:Bachir With His Father Pierre (...
en.wikipedia.org (parse) Assassination of Iranian nuclear scientists
en.wikipedia.org (parse) Assassination of René Moawad
en.wikipedia.org (imageinfo) File:René Moawad.jpg
en.wikipedia.org (parse) Assem Qanso
en.wikipedia.org (imageinfo) File:Assem Qanso with Nicolae Ceaușe...
en.wikipedia.org (parse) At Tiri incident
en.wikipedia.org (parse) Axis of Resistance
en.wikipedia.org (parse) Ayatollah Khomeini
en.wikipedia.org (imageinfo) File:Ruhollah Khomeini portrait 1.jpg
en.wikipedia.org (parse) Ba'athism
en.wikipedia.org (parse) Ba'athist Iraq
en.wikipedia.org (parse) Baabda
en.wikipedia.org (parse) Baalbeck
en.wikipedia.org (parse) Bab al-Tabbaneh–Jabal Mohsen conflict
en.wikipedia.org (imageinfo) File:Lebanese army on Syria Street.jpg
en.wikipedia.org (parse) Bachir Gemayel
en.wikipedia.org (imageinfo) File:Bachir With H

LookupError: https://en.wikipedia.org/w/api.php?action=parse&formatversion=2&contentmodel=text&disableeditsection=&disablelimitreport=&disabletoc=&prop=text|iwlinks|parsetree|wikitext|displaytitle|properties&redirects&page=Bibliography%20of%20Lebanon